In [18]:
#libraries
import numpy as np
import random
import math
import cv2
import Queue
from collections import defaultdict
from collections import deque
import time

In [19]:
####ALL FUNCTIONS GO HERE####

#####FUNCTIONS FOR PATHFINDING######

# maze to graph converts 2d array to a graph
# taken from http://bryukh.com/labyrinth-algorithms/
def maze2graph(maze):
    
    print "Started maze2graph"
    startTime = time.time()
    
    height = len(maze)
    width = len(maze[0]) if height else 0
    graph = {(i, j): [] for j in range(width) for i in range(height) if not maze[i][j]}
    for row, col in graph.keys():
        if row < height - 1 and not maze[row + 1][col]:
            graph[(row, col)].append(("S", (row + 1, col)))
            graph[(row + 1, col)].append(("N", (row, col)))
        if col < width - 1 and not maze[row][col + 1]:
            graph[(row, col)].append(("E", (row, col + 1)))
            graph[(row, col + 1)].append(("W", (row, col)))
            
    print "Done maze2graph, took", time.time() - startTime, "seconds."
    
    return graph

#findpathbfs uses bredth first search to find a path from start to finish
# taken from http://bryukh.com/labyrinth-algorithms/
def find_path_bfs(maze, start, goal):
    
    print "Started find_path_bfs"
    startTime = time.time()
    
    queue = deque([("", start)])
    visited = set()
    graph = maze2graph(maze)
    while queue:
        path, current = queue.popleft()
        if current == goal:
            
            print "Done find_path_bfs, took", time.time() - startTime, "seconds."
            
            return path
        if current in visited:
            continue
        visited.add(current)
        for direction, neighbour in graph[current]:
            queue.append((path + direction, neighbour))
    return "NO WAY!"

#drawPath takes in an image and draws the path on it
def drawPath(image, direct, start):
    
    #creates blank image
    image = np.zeros((image.shape[0], image.shape[1], 3),np.uint8)
    
    colour = [random.randint(0,255),random.randint(0,255),random.randint(0,255)]
    
    current = (start[0]+ 100, start[1] + 200)
    
    for i in range(0, len(direct)):
        image[current] = colour
        
        #colourSpot(image, current[0], current[1], colour)
        
        if direct[i] == "N":
            current = (current[0]-1,current[1])
        if direct[i] == "S":
            current = (current[0]+1,current[1])
        if direct[i] == "E":
            current = (current[0],current[1]+1)
        if direct[i] == "W":
            current = (current[0],current[1]-1)
            
    return image

# creates maze using noise, then finds a path between two random points 
# and returns the path on a black bg
def generatePath(img):
    
    print "Started generatePath"
    startTime = time.time()

    #creates an array of noise, image is smaller than whole to save time/centre line
    maze = np.zeros((img.shape[0] - 200 , img.shape[1] - 400))

    for row in range(0, maze.shape[0]):
        for col in range(0, maze.shape[1]):
             maze[row][col] = random.choice([0, 0, 1])
                
    print img.shape            
    print maze.shape
    
    
    #creates points in maze
    start = (random.randint(0, maze.shape[0]-1), random.randint(0, maze.shape[1]-1))
    end = (random.randint(0, maze.shape[0]-1), random.randint(0 , maze.shape[1]-1))

    #makes sure that start and end aren't in obsticles
    while maze[start] or maze[end] == 1:
        start = (random.randint(0, maze.shape[0]-1), random.randint(0, maze.shape[1]-1))
        end = (random.randint(0, maze.shape[0]-1), random.randint(0 , maze.shape[1]-1))

    #creates and fills an array with the path
    direct = []
    direct = find_path_bfs(maze, start, end)

    #draws the path on a blank picture
    line = drawPath(img, direct, start)
    
    print "Done generatePath, took", time.time() - startTime, "seconds."
    
    return line

#### OTHER FUNCTONS ######

# puts a noise filter over a image and returns the result
def drugifyFast(image):
    
    print "Started drugifyFast"
    startTime = time.time()
    
    values = [[0,0,0],[0,0,255],[0,255,0],[0,255,255],[255,0,0],[255,0,255],[255,255,0],[255,255,255]]
    
    noise = np.zeros((image.shape[0],image.shape[1],3),np.uint8)
    
    for row in range(0, noise.shape[0]):
        for col in range(0, noise.shape[1]):
                noise[row][col] = values[random.randint(0,7)]
                
    image = cv2.addWeighted(noise, .65, image, .35, 0)
    
    print "Done drugifyFast, took", time.time() - startTime, "seconds."
    
    return image

#colours a certain pixel area around a certain point
def colourSpot(image, row, col, colour):
    for i in range(-1, 2):
        for j in range(-1, 2):
            image[row+i, col+j] = colour
        
#fades the right and left of a border higher num is less dark
def fadeEdges(image):
    
    startTime = time.time()
    
    print "Started fadeEdges"
    
    centre = image.shape[1]//2
    
    for i in range(0, image.shape[1]):
        dist = abs(centre - i) + 1
        val = dist/2.7
        
        if dist < 30:
            val = 15
        
        col = image[:,i,:]
        
        image[:,i,:] = cv2.addWeighted(col, 1, col, 0, -val)
                    
    print "Done fadeEdges, took", time.time() - startTime, "seconds."

    return image


In [20]:
def filterFrame(img):
    print "Started imageProcess"
    Time = time.time()

    img = drugifyFast(img)

    line = generatePath(img)

    img = cv2.addWeighted(img, 1, line, 1, -30)

    img = fadeEdges(img)

    print "Done imageProcess, took", time.time() - Time, "seconds in total."
    
    return img

img = cv2.imread('frame.jpg')

img = filterFrame(img)

cv2.imshow('frame', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

Started imageProcess
Started drugifyFast
Done drugifyFast, took 2.39299988747 seconds.
Started generatePath
(720L, 1280L, 3L)
(520L, 880L)
Started find_path_bfs
Started maze2graph
Done maze2graph, took 1.66799998283 seconds.
Done find_path_bfs, took 2.25600004196 seconds.
Done generatePath, took 3.02700018883 seconds.
Started fadeEdges
Done fadeEdges, took 0.0329999923706 seconds.
Done imageProcess, took 5.45700001717 seconds in total.


In [21]:
#start of video editing

cap = cv2.VideoCapture('apartment.avi')

count = 0
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret==True:
        frame = filterFrame(frame)

        cv2.imwrite('vid' + str(count) + '.png',frame)
        count+=1
        
        #cv2.imshow('frame',frame)
        #if cv2.waitKey(1) & 0xFF == ord('q'):
        #    break
    else:
        break

# Release everything if job is finished
cap.release()
cv2.destroyAllWindows()

Started imageProcess
Started drugifyFast
Done drugifyFast, took 2.45600008965 seconds.
Started generatePath
(720L, 1280L, 3L)
(520L, 880L)
Started find_path_bfs
Started maze2graph
Done maze2graph, took 1.51900005341 seconds.
Done find_path_bfs, took 2.2009999752 seconds.
Done generatePath, took 3.01200008392 seconds.
Started fadeEdges
Done fadeEdges, took 0.0309998989105 seconds.
Done imageProcess, took 5.50400018692 seconds in total.
Started imageProcess
Started drugifyFast
Done drugifyFast, took 2.42700004578 seconds.
Started generatePath
(720L, 1280L, 3L)
(520L, 880L)
Started find_path_bfs
Started maze2graph
Done maze2graph, took 1.51999998093 seconds.
Done generatePath, took 3.06200003624 seconds.
Started fadeEdges
Done fadeEdges, took 0.0320000648499 seconds.
Done imageProcess, took 5.52499985695 seconds in total.
Started imageProcess
Started drugifyFast
Done drugifyFast, took 2.45300006866 seconds.
Started generatePath
(720L, 1280L, 3L)
(520L, 880L)
Started find_path_bfs
Started 

Done drugifyFast, took 2.41900014877 seconds.
Started generatePath
(720L, 1280L, 3L)
(520L, 880L)
Started find_path_bfs
Started maze2graph
Done maze2graph, took 1.52699995041 seconds.
Done find_path_bfs, took 1.84399986267 seconds.
Done generatePath, took 2.61699986458 seconds.
Started fadeEdges
Done fadeEdges, took 0.0310001373291 seconds.
Done imageProcess, took 5.07100009918 seconds in total.
Started imageProcess
Started drugifyFast
Done drugifyFast, took 2.40600013733 seconds.
Started generatePath
(720L, 1280L, 3L)
(520L, 880L)
Started find_path_bfs
Started maze2graph
Done maze2graph, took 1.53900003433 seconds.
Done find_path_bfs, took 1.5720000267 seconds.
Done generatePath, took 2.34699988365 seconds.
Started fadeEdges
Done fadeEdges, took 0.0310001373291 seconds.
Done imageProcess, took 4.78800010681 seconds in total.
Started imageProcess
Started drugifyFast
Done drugifyFast, took 2.44299983978 seconds.
Started generatePath
(720L, 1280L, 3L)
(520L, 880L)
Started find_path_bfs
S

Done drugifyFast, took 2.52399992943 seconds.
Started generatePath
(720L, 1280L, 3L)
(520L, 880L)
Started find_path_bfs
Started maze2graph
Done maze2graph, took 1.64299988747 seconds.
Done find_path_bfs, took 2.14599990845 seconds.
Done generatePath, took 2.992000103 seconds.
Started fadeEdges
Done fadeEdges, took 0.0450000762939 seconds.
Done imageProcess, took 5.56900000572 seconds in total.
Started imageProcess
Started drugifyFast
Done drugifyFast, took 2.4470000267 seconds.
Started generatePath
(720L, 1280L, 3L)
(520L, 880L)
Started find_path_bfs
Started maze2graph
Done maze2graph, took 1.62300014496 seconds.
Done find_path_bfs, took 1.6890001297 seconds.
Done generatePath, took 2.51999998093 seconds.
Started fadeEdges
Done fadeEdges, took 0.0409998893738 seconds.
Done imageProcess, took 5.01200008392 seconds in total.
Started imageProcess
Started drugifyFast
Done drugifyFast, took 2.42999982834 seconds.
Started generatePath
(720L, 1280L, 3L)
(520L, 880L)
Started find_path_bfs
Star

Done drugifyFast, took 2.43499994278 seconds.
Started generatePath
(720L, 1280L, 3L)
(520L, 880L)
Started find_path_bfs
Started maze2graph
Done maze2graph, took 1.59500002861 seconds.
Done find_path_bfs, took 2.24800014496 seconds.
Done generatePath, took 3.05999994278 seconds.
Started fadeEdges
Done fadeEdges, took 0.0299999713898 seconds.
Done imageProcess, took 5.52999997139 seconds in total.
Started imageProcess
Started drugifyFast
Done drugifyFast, took 2.46900010109 seconds.
Started generatePath
(720L, 1280L, 3L)
(520L, 880L)
Started find_path_bfs
Started maze2graph
Done maze2graph, took 1.59500002861 seconds.
Done find_path_bfs, took 2.23300004005 seconds.
Done generatePath, took 3.0680000782 seconds.
Started fadeEdges
Done fadeEdges, took 0.0310001373291 seconds.
Done imageProcess, took 5.57300019264 seconds in total.
Started imageProcess
Started drugifyFast
Done drugifyFast, took 2.49300003052 seconds.
Started generatePath
(720L, 1280L, 3L)
(520L, 880L)
Started find_path_bfs
S

Done drugifyFast, took 2.50999999046 seconds.
Started generatePath
(720L, 1280L, 3L)
(520L, 880L)
Started find_path_bfs
Started maze2graph
Done maze2graph, took 1.71000003815 seconds.
Done find_path_bfs, took 1.74900007248 seconds.
Done generatePath, took 2.5720000267 seconds.
Started fadeEdges
Done fadeEdges, took 0.0320000648499 seconds.
Done imageProcess, took 5.11899995804 seconds in total.
Started imageProcess
Started drugifyFast
Done drugifyFast, took 2.43700003624 seconds.
Started generatePath
(720L, 1280L, 3L)
(520L, 880L)
Started find_path_bfs
Started maze2graph
Done maze2graph, took 1.55900001526 seconds.
Done generatePath, took 3.14899992943 seconds.
Started fadeEdges
Done fadeEdges, took 0.0320000648499 seconds.
Done imageProcess, took 5.62199997902 seconds in total.
Started imageProcess
Started drugifyFast
Done drugifyFast, took 2.52300000191 seconds.
Started generatePath
(720L, 1280L, 3L)
(520L, 880L)
Started find_path_bfs
Started maze2graph
Done maze2graph, took 1.585999

...

Done drugifyFast, took 2.46300005913 seconds.
Started generatePath
(720L, 1280L, 3L)
(520L, 880L)
Started find_path_bfs
Started maze2graph
Done maze2graph, took 1.47499990463 seconds.
Done find_path_bfs, took 1.85699987411 seconds.
Done generatePath, took 2.64600014687 seconds.
Started fadeEdges
Done fadeEdges, took 0.0309998989105 seconds.
Done imageProcess, took 5.14499998093 seconds in total.
Started imageProcess
Started drugifyFast
Done drugifyFast, took 2.42499995232 seconds.
Started generatePath
(720L, 1280L, 3L)
(520L, 880L)
Started find_path_bfs
Started maze2graph
Done maze2graph, took 1.6369998455 seconds.
Done find_path_bfs, took 2.10799980164 seconds.
Done generatePath, took 2.8900001049 seconds.
Started fadeEdges
Done fadeEdges, took 0.0309998989105 seconds.
Done imageProcess, took 5.35299992561 seconds in total.
Started imageProcess
Started drugifyFast
Done drugifyFast, took 2.43799996376 seconds.
Started generatePath
(720L, 1280L, 3L)
(520L, 880L)
Started find_path_bfs
St

Done drugifyFast, took 2.46799993515 seconds.
Started generatePath
(720L, 1280L, 3L)
(520L, 880L)
Started find_path_bfs
Started maze2graph
Done maze2graph, took 1.59299993515 seconds.
Done find_path_bfs, took 1.99699997902 seconds.
Done generatePath, took 2.77900004387 seconds.
Started fadeEdges
Done fadeEdges, took 0.0289998054504 seconds.
Done imageProcess, took 5.28100013733 seconds in total.
Started imageProcess
Started drugifyFast
Done drugifyFast, took 2.43099999428 seconds.
Started generatePath
(720L, 1280L, 3L)
(520L, 880L)
Started find_path_bfs
Started maze2graph
Done maze2graph, took 1.61099982262 seconds.
Done find_path_bfs, took 2.007999897 seconds.
Done generatePath, took 2.78299999237 seconds.
Started fadeEdges
Done fadeEdges, took 0.0300002098083 seconds.
Done imageProcess, took 5.24800014496 seconds in total.
Started imageProcess
Started drugifyFast
Done drugifyFast, took 2.48099994659 seconds.
Started generatePath
(720L, 1280L, 3L)
(520L, 880L)
Started find_path_bfs
St

Done drugifyFast, took 2.46099996567 seconds.
Started generatePath
(720L, 1280L, 3L)
(520L, 880L)
Started find_path_bfs
Started maze2graph
Done maze2graph, took 1.43499994278 seconds.
Done generatePath, took 2.242000103 seconds.
Started fadeEdges
Done fadeEdges, took 0.0299999713898 seconds.
Done imageProcess, took 4.73900008202 seconds in total.
Started imageProcess
Started drugifyFast
Done drugifyFast, took 2.43400001526 seconds.
Started generatePath
(720L, 1280L, 3L)
(520L, 880L)
Started find_path_bfs
Started maze2graph
Done maze2graph, took 1.6369998455 seconds.
Done find_path_bfs, took 2.06799983978 seconds.
Done generatePath, took 2.86199998856 seconds.
Started fadeEdges
Done fadeEdges, took 0.0309998989105 seconds.
Done imageProcess, took 5.32999992371 seconds in total.
Started imageProcess
Started drugifyFast
Done drugifyFast, took 2.45599985123 seconds.
Started generatePath
(720L, 1280L, 3L)
(520L, 880L)
Started find_path_bfs
Started maze2graph
Done maze2graph, took 1.44600009

Done drugifyFast, took 2.4470000267 seconds.
Started generatePath
(720L, 1280L, 3L)
(520L, 880L)
Started find_path_bfs
Started maze2graph
Done maze2graph, took 1.39800000191 seconds.
Done find_path_bfs, took 1.62199997902 seconds.
Done generatePath, took 2.40799999237 seconds.
Started fadeEdges
Done fadeEdges, took 0.0300002098083 seconds.
Done imageProcess, took 4.88900017738 seconds in total.
Started imageProcess
Started drugifyFast
Done drugifyFast, took 2.45199990273 seconds.
Started generatePath
(720L, 1280L, 3L)
(520L, 880L)
Started find_path_bfs
Started maze2graph
Done maze2graph, took 1.63800001144 seconds.
Done find_path_bfs, took 1.78299999237 seconds.
Done generatePath, took 2.57700014114 seconds.
Started fadeEdges
Done fadeEdges, took 0.0299999713898 seconds.
Done imageProcess, took 5.06399989128 seconds in total.
Started imageProcess
Started drugifyFast
Done drugifyFast, took 2.46399998665 seconds.
Started generatePath
(720L, 1280L, 3L)
(520L, 880L)
Started find_path_bfs
S

Done drugifyFast, took 2.45600008965 seconds.
Started generatePath
(720L, 1280L, 3L)
(520L, 880L)
Started find_path_bfs
Started maze2graph
Done maze2graph, took 1.57299995422 seconds.
Done find_path_bfs, took 1.94299983978 seconds.
Done generatePath, took 2.78200006485 seconds.
Started fadeEdges
Done fadeEdges, took 0.0310001373291 seconds.
Done imageProcess, took 5.27500009537 seconds in total.
Started imageProcess
Started drugifyFast
Done drugifyFast, took 2.43700003624 seconds.
Started generatePath
(720L, 1280L, 3L)
(520L, 880L)
Started find_path_bfs
Started maze2graph
Done maze2graph, took 1.59399986267 seconds.
Done find_path_bfs, took 1.99100017548 seconds.
Done generatePath, took 2.80400013924 seconds.
Started fadeEdges
Done fadeEdges, took 0.0320000648499 seconds.
Done imageProcess, took 5.27700018883 seconds in total.
Started imageProcess
Started drugifyFast
Done drugifyFast, took 2.50699996948 seconds.
Started generatePath
(720L, 1280L, 3L)
(520L, 880L)
Started find_path_bfs
